# MicroStrateg(P)ython Migration

## Load libraries

In [1]:
import json
from mstrio.object_management import folder
import pandas as pd
from time import sleep
import shutil
import os
from mstrio.connection import Connection
from mstrio.object_management.migration import (
    PackageType,
    Migration,
    PackageConfig,
    PackageSettings
)

from mstrio.object_management.migration.package import (
    Action,
    ImportStatus,
    PackageStatus,
    ValidationStatus,
)



def bld_mig_list_from_sh_fold(conn, folder_id,action= "USE_EXISTING",include_dependents= False ):

    #obj_l = get_folder_obj_l(conn=conn, folder_id=folder_id)
    i_folder = folder.Folder(connection=conn, id=folder_id)
    obj_l = i_folder.get_contents(to_dictionary=True)
    
    obj_det_l=[]
    for obj_d in obj_l:
        #print(obj_d)
        if obj_d["type"] == 18:
            obj_det_d = {}
            obj_det_d["id"] = obj_d["target_info"]["id"]
            obj_det_d["type"] = obj_d["target_info"]["type"]
            obj_det_d["action"] = action
            obj_det_d["include_dependents"] = include_dependents
            obj_det_d["subtype"] = obj_d["target_info"]["subtype"] # only for read out object definition (see end of note book)
            obj_det_l.append(obj_det_d.copy())  

    return obj_det_l


In [2]:
from mstrio.connection import Connection
with open('..\\config\\user_d.json', 'r') as openfile:
    user_d = json.load(openfile)

conn_params =  user_d["conn_params"]
source_conn = Connection(**conn_params)

Connection to MicroStrategy Intelligence Server has been established.
No project selected.


## Configure Migration

In [3]:
#define Package
usr_migration_folder_name="Daniel_Migration"
usr_package_folder_name="20250223_demo"


short_cut_folder_id="986E6D1A427030A710A0E68F74290482"

# Connect Projects
with open('..\\config\\user_d.json', 'r') as openfile:
    user_d = json.load(openfile)
conn_params =  user_d["conn_params"]
#set user credentials and open a connection to the i-server


source_project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
target_project_id="00F3103E41FF3743A057C3B5313595B7"
package_settings = PackageSettings(
    Action.FORCE_REPLACE,
    PackageSettings.UpdateSchema.RECAL_TABLE_LOGICAL_SIZE,
    PackageSettings.AclOnReplacingObjects.REPLACE,
    PackageSettings.AclOnNewObjects.KEEP_ACL_AS_SOURCE_OBJECT,
)

## Connect to projects

In [4]:
conn_params["project_id"]=source_project_id
#conn = Connection(**conn_params)
source_conn = Connection(**conn_params)
#source_conn.headers['Content-type'] = "application/json"


conn_params["project_id"]=target_project_id
target_conn = Connection(**conn_params)

package_name=usr_package_folder_name
mstr_mig_folder=source_conn.environment.storage_service.location 
migration_folder= source_conn.environment.storage_service.location  +"\\"+ usr_migration_folder_name +"\\"+ usr_package_folder_name + "\\"
migration_folder


Connection to MicroStrategy Intelligence Server has been established.
Connection to MicroStrategy Intelligence Server has been established.


'D:\\shared_drive\\MSTR_Migrations\\Daniel_Migration\\20250223_demo\\'

## Create package

In [5]:
package_content_info=bld_mig_list_from_sh_fold(conn=source_conn, folder_id=short_cut_folder_id,action= "FORCE_REPLACE",include_dependents= True)
package_config = PackageConfig(
    package_settings,  package_content_info # [ package_content_from_object]
)

my_obj_mig = Migration.create_object_migration(
    connection=source_conn,
    toc_view=package_config,
    name=package_name,
    project_id=source_conn.project_id,
)

while my_obj_mig.package_info.status == PackageStatus.CREATING:
    sleep(2)
    my_obj_mig.fetch()

my_obj_mig.package_info.status

Folder object named: '20250123_Demo' with ID: '986E6D1A427030A710A0E68F74290482'
Successfully started creation of migration object with ID: 'E735981A4EFE44CC89B637A8D46434FD:62EEDF4F4F9F413A98A3A85DB419AD55'


PackageStatus.CREATED

In [6]:
my_obj_mig.id

'E735981A4EFE44CC89B637A8D46434FD:62EEDF4F4F9F413A98A3A85DB419AD55'

om_pack_d["packageInfo"]

## Validate Package

In [ ]:
# Check migration validation status
my_obj_mig.trigger_validation(
    target_env=target_conn, target_project_name=target_conn.project_name
)

while my_obj_mig.validation.status == ValidationStatus.VALIDATING:
    sleep(2)
    my_obj_mig.fetch()


print(my_obj_mig.validation)

## Move Migration Pack to migration folder

In [ ]:
package_file = mstr_mig_folder +"/" +my_obj_mig.package_info.storage.path
package_file=os.path.normpath(package_file)
my_obj_mig_path=shutil.move(package_file, migration_folder)

print("org_file: " +package_file)
print("migration folder: " +my_obj_mig_path)


## Trigger migration

In [ ]:
my_obj_mig_path="D:\\shared_drive\\MSTR_Migrations\\demo\\simpleMetric.mmp"
from_file_mig = Migration.migrate_from_file(
    connection=target_conn,
    file_path=my_obj_mig_path,
    package_type=PackageType.OBJECT,
    name="object_mig_from_local_storage",
    target_project_name=target_conn.project_name
)

while from_file_mig.import_info.status == ImportStatus.IMPORTING:
    sleep(2)
    from_file_mig.fetch()

from_file_mig.import_info.status

In [ ]:
from_file_mig.import_info

## Archive Migration

### move undo mmp file to migration folder

In [ ]:
## move undo mmp to migration folder
undo_source_file = mstr_mig_folder +"/" +from_file_mig.import_info.undo_storage.path
package_file=os.path.normpath(package_file)
new_undo_file=shutil.move(undo_source_file, migration_folder+"Undo.mmp")

print("org_undo_file: " +undo_source_file)
print("migration folder: " +new_undo_file)

### store migrated objects to CSV file

In [ ]:
package_content_df=pd.DataFrame(package_content_info)
package_content_df.to_csv(migration_folder+"package_content.csv")
package_content_df

### save package & migration files

In [ ]:
file_path=migration_folder+"package_info.txt"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(str(my_obj_mig.package_info))

file_path=migration_folder+"migration_info.txt"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(str(from_file_mig.import_info))


## Read & Store Object definition

In [ ]:
from mstr_robotics.read_out_prj_obj import read_gen
i_read_gen=read_gen()

obj_def_l=[]
for obj in package_content_info:
    obj_def=i_read_gen.get_obj_def(conn=source_conn,object_id=obj["id"],obj_type=obj["type"],obj_sub_type=obj["subtype"])
    obj_def_l.append(obj_def)

file_path=migration_folder+"object_def.json"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(json.dumps(obj_def_l))
    
len(obj_def_l)

In [ ]:
source_conn.close()
target_conn.close()

## Run undo

In [ ]:
from_file_mig = Migration.migrate_from_file(
    connection=target_conn,
    file_path=new_undo_file,
    package_type=PackageType.OBJECT,
    name="object_mig_from_local_storage",
    target_project_name=target_conn.project_name,
)

while from_file_mig.import_info.status == ImportStatus.IMPORTING:
    sleep(2)
    from_file_mig.fetch()

from_file_mig.import_info.status